In [1]:
import ibis
import ibis_ml as ml
from ibis import _

ibis.options.interactive = True

In [2]:
model_input_table = ibis.read_parquet("../solutions/small_model_input_table.parquet")
model_input_table

┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ game_id  ┃ is_rated ┃ lichess_time_control_type ┃ is_tournament ┃ white_elo ┃ black_elo ┃ white_title ┃ black_title ┃ base_time ┃ increment ┃ target  ┃ ply   ┃ mate_eval ┃ regular_eval ┃ white_clock ┃ black_clock ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ boolean  │ string                    │ boolean       │ int64     │ int64     │ string      │ string      │ int64     │ int64     │ float64 │ int64 │ int64     │ float64      │ float64     │ float64     │
├──────────┼──────────┼───────────────────────────┼───────────────┼───────────┼───────────┼─────────────┼─────────────┼───────────┼───────────┼─────────┼───────┼───────────┼──────────────┼─────────────┼─────────────┤
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │     1 │      NULL │         0.18 │       600.0 │       600.0 │
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │     2 │      NULL │         0.72 │       600.0 │       600.0 │
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │     3 │      NULL │         0.70 │       597.0 │       600.0 │
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │     4 │      NULL │         0.63 │       597.0 │       600.0 │
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │     5 │      NULL │         0.37 │       596.0 │       600.0 │
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │     6 │      NULL │         0.40 │       596.0 │       595.0 │
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │     7 │      NULL │         0.38 │       595.0 │       595.0 │
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │     8 │      NULL │         0.30 │       595.0 │       584.0 │
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │     9 │      NULL │         0.18 │       593.0 │       584.0 │
│ 3ZR2DrL8 │ True     │ Rapid                     │ False         │      2026 │      2077 │ NULL        │ NULL        │       600 │         0 │     0.0 │    10 │      NULL │         0.95 │       593.0 │       570.0 │
│ …        │ …        │ …                         │ …             │         … │         … │ …           │ …           │         … │         … │       … │     … │         … │            … │           … │           … │
└──────────┴──────────┴───────────────────────────┴───────────────┴───────────┴───────────┴─────────────┴─────────────┴───────────┴───────────┴─────────┴───────┴───────────┴──────────────┴─────────────┴─────────────┘

# Data splitting

In [3]:
# Create data frames for the two sets:
train_data, test_data = ml.train_test_split(
    model_input_table,
    unique_key="game_id",
    # Put 3/4 of the data into the training set
    test_size=0.25,
    num_buckets=4,
    # Set the seed to enable reproducible analysis
    random_seed=111,
)

# Fit a model with a recipe

In [4]:
import xgboost as xgb
from sklearn.pipeline import Pipeline

NUM_MOVES = 40

X_train = train_data.drop("target")
y_train = train_data.target

xgb_reg_pipe = Pipeline(
    [
        (
            "xgb_recipe",
            ml.Recipe(
                ml.Mutate(
                    relative_clock_diff=(_.white_clock - _.black_clock)
                    / (_.base_time + _.increment * NUM_MOVES)
                ),
                ml.DropZeroVariance(ml.everything()),
                ml.Drop(ml.string()),
                ml.Cast(ml.everything(), "float64"),
            ),
        ),
        ("xgb_reg", xgb.XGBRegressor(n_estimators=10)),
    ]
)
xgb_reg_pipe.set_output(transform="pandas")
# The regressor seems to overfit much more quickly than the classifier
# With the default eta=0.3, try n_estimators=10
# With eta=0.1, try n_estimators=20
xgb_reg_pipe.fit(X_train, y_train)

xgb_reg_pipe["xgb_reg"].save_model("live-win-probability-model.json")

# Perform inference using LETSQL

In [5]:
import letsql as ls

import ibis.selectors as s
from letsql.common.caching import ParquetCacheStorage

con = ls.connect()

duckdb_con = ls.duckdb.connect()
cache = ParquetCacheStorage(source=con, path="letsql-tmp")

t = (
    duckdb_con.read_parquet("../solutions/small_model_input_table.parquet")
    .pipe(con.register, "model_input_table")
    .cache(storage=cache)
)
model_path = "live-win-probability-model.json"

predict_xgb = con.register_xgb_model("live_win_probability", model_path)

In [6]:
t

┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ game_id  ┃ is_rated ┃ lichess_time_control_type ┃ is_tournament ┃ white_elo ┃ black_elo ┃ white_title ┃ black_title ┃ base_time ┃ increment ┃ target  ┃ ply   ┃ mate_eval ┃ regular_eval ┃ white_clock ┃ black_clock ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ boolean  │ string                    │ boolean       │ int64     │ int64     │ string      │ string      │ int64     │ int64     │ float64 │ int64 │ int64     │ float64      │ float64     │ float64     │
├──────────┼──────────┼───────────────────────────┼───────────────┼───────────┼───────────┼─────────────┼─────────────┼───────────┼───────────┼─────────┼───────┼───────────┼──────────────┼─────────────┼─────────────┤
│ 442d9Ayn │ True     │ Rapid                     │ False         │      1433 │      1500 │ NULL        │ NULL        │       600 │         0 │     0.0 │    83 │      -998 │         NULL │       199.0 │       326.0 │
│ 442d9Ayn │ True     │ Rapid                     │ False         │      1433 │      1500 │ NULL        │ NULL        │       600 │         0 │     0.0 │    84 │      NULL │         6.41 │       199.0 │       319.0 │
│ 442d9Ayn │ True     │ Rapid                     │ False         │      1433 │      1500 │ NULL        │ NULL        │       600 │         0 │     0.0 │    85 │      -999 │         NULL │       194.0 │       319.0 │
│ 442d9Ayn │ True     │ Rapid                     │ False         │      1433 │      1500 │ NULL        │ NULL        │       600 │         0 │     0.0 │    86 │     -1000 │         NULL │       194.0 │       318.0 │
│ 9lFAN0gK │ True     │ Blitz                     │ False         │      1229 │      1236 │ NULL        │ NULL        │       300 │         0 │     0.0 │     1 │      NULL │         0.18 │       300.0 │       300.0 │
│ 9lFAN0gK │ True     │ Blitz                     │ False         │      1229 │      1236 │ NULL        │ NULL        │       300 │         0 │     0.0 │     3 │      NULL │         0.21 │       297.0 │       300.0 │
│ 9lFAN0gK │ True     │ Blitz                     │ False         │      1229 │      1236 │ NULL        │ NULL        │       300 │         0 │     0.0 │     4 │      NULL │         0.53 │       297.0 │       298.0 │
│ 9lFAN0gK │ True     │ Blitz                     │ False         │      1229 │      1236 │ NULL        │ NULL        │       300 │         0 │     0.0 │     6 │      NULL │         0.67 │       297.0 │       295.0 │
│ 9lFAN0gK │ True     │ Blitz                     │ False         │      1229 │      1236 │ NULL        │ NULL        │       300 │         0 │     0.0 │     7 │      NULL │         0.63 │       294.0 │       295.0 │
│ 9lFAN0gK │ True     │ Blitz                     │ False         │      1229 │      1236 │ NULL        │ NULL        │       300 │         0 │     0.0 │     8 │      NULL │         0.78 │       294.0 │       293.0 │
│ …        │ …        │ …                         │ …             │         … │         … │ …           │ …           │         … │         … │       … │     … │         … │            … │           … │           … │
└──────────┴──────────┴───────────────────────────┴───────────────┴───────────┴───────────┴─────────────┴─────────────┴───────────┴───────────┴─────────┴───────┴───────────┴──────────────┴─────────────┴─────────────┘

In [7]:
xgb_reg_pipe["xgb_recipe"].to_ibis(t)

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ is_tournament ┃ white_elo ┃ black_elo ┃ base_time ┃ increment ┃ target  ┃ ply     ┃ mate_eval ┃ regular_eval ┃ white_clock ┃ black_clock ┃ relative_clock_diff ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ float64       │ float64   │ float64   │ float64   │ float64   │ float64 │ float64 │ float64   │ float64      │ float64     │ float64     │ float64             │
├───────────────┼───────────┼───────────┼───────────┼───────────┼─────────┼─────────┼───────────┼──────────────┼─────────────┼─────────────┼─────────────────────┤
│           0.0 │    1433.0 │    1500.0 │     600.0 │       0.0 │     0.0 │    83.0 │    -998.0 │         NULL │       199.0 │       326.0 │           -0.211667 │
│           0.0 │    1433.0 │    1500.0 │     600.0 │       0.0 │     0.0 │    84.0 │      NULL │         6.41 │       199.0 │       319.0 │           -0.200000 │
│           0.0 │    1433.0 │    1500.0 │     600.0 │       0.0 │     0.0 │    85.0 │    -999.0 │         NULL │       194.0 │       319.0 │           -0.208333 │
│           0.0 │    1433.0 │    1500.0 │     600.0 │       0.0 │     0.0 │    86.0 │   -1000.0 │         NULL │       194.0 │       318.0 │           -0.206667 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     1.0 │      NULL │         0.18 │       300.0 │       300.0 │            0.000000 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     3.0 │      NULL │         0.21 │       297.0 │       300.0 │           -0.010000 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     4.0 │      NULL │         0.53 │       297.0 │       298.0 │           -0.003333 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     6.0 │      NULL │         0.67 │       297.0 │       295.0 │            0.006667 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     7.0 │      NULL │         0.63 │       294.0 │       295.0 │           -0.003333 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     8.0 │      NULL │         0.78 │       294.0 │       293.0 │            0.003333 │
│             … │         … │         … │         … │         … │       … │       … │         … │            … │           … │           … │                   … │
└───────────────┴───────────┴───────────┴───────────┴───────────┴─────────┴─────────┴───────────┴──────────────┴─────────────┴─────────────┴─────────────────────┘

In [8]:
t_pred = (
    xgb_reg_pipe["xgb_recipe"]
    .to_ibis(t)
    .mutate(predicted_live_win_probability=predict_xgb.on_expr)
)
t_pred

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ is_tournament ┃ white_elo ┃ black_elo ┃ base_time ┃ increment ┃ target  ┃ ply     ┃ mate_eval ┃ regular_eval ┃ white_clock ┃ black_clock ┃ relative_clock_diff ┃ predicted_live_win_probability ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ float64       │ float64   │ float64   │ float64   │ float64   │ float64 │ float64 │ float64   │ float64      │ float64     │ float64     │ float64             │ float64                        │
├───────────────┼───────────┼───────────┼───────────┼───────────┼─────────┼─────────┼───────────┼──────────────┼─────────────┼─────────────┼─────────────────────┼────────────────────────────────┤
│           0.0 │    1433.0 │    1500.0 │     600.0 │       0.0 │     0.0 │    83.0 │    -998.0 │         NULL │       199.0 │       326.0 │           -0.211667 │                       0.098014 │
│           0.0 │    1433.0 │    1500.0 │     600.0 │       0.0 │     0.0 │    84.0 │      NULL │         6.41 │       199.0 │       319.0 │           -0.200000 │                       0.157806 │
│           0.0 │    1433.0 │    1500.0 │     600.0 │       0.0 │     0.0 │    85.0 │    -999.0 │         NULL │       194.0 │       319.0 │           -0.208333 │                       0.072440 │
│           0.0 │    1433.0 │    1500.0 │     600.0 │       0.0 │     0.0 │    86.0 │   -1000.0 │         NULL │       194.0 │       318.0 │           -0.206667 │                       0.049347 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     1.0 │      NULL │         0.18 │       300.0 │       300.0 │            0.000000 │                       0.125494 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     3.0 │      NULL │         0.21 │       297.0 │       300.0 │           -0.010000 │                       0.125494 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     4.0 │      NULL │         0.53 │       297.0 │       298.0 │           -0.003333 │                       0.136901 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     6.0 │      NULL │         0.67 │       297.0 │       295.0 │            0.006667 │                       0.136901 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     7.0 │      NULL │         0.63 │       294.0 │       295.0 │           -0.003333 │                       0.136901 │
│           0.0 │    1229.0 │    1236.0 │     300.0 │       0.0 │     0.0 │     8.0 │      NULL │         0.78 │       294.0 │       293.0 │            0.003333 │                       0.136901 │
│             … │         … │         … │         … │         … │       … │       … │         … │            … │           … │           … │                   … │                              … │
└───────────────┴───────────┴───────────┴───────────┴───────────┴─────────┴─────────┴───────────┴──────────────┴─────────────┴─────────────┴─────────────────────┴────────────────────────────────┘